In [2]:
import sys
import os
import base64
import subprocess
import random
import math
import aes
import telnet_client
import netstring
import pkcs1
import itertools
#from CryptoPublicKey import RSA
#"pk=RSA.importKey(B)"
#"pk.N / pk.e"

In [3]:
#!{sys.executable} -m pip install --user twisted

In [9]:
#Résolution de challenge

challenge = "sexes washy bolls litho cider"
fic = open("challenge","w")
fic.write(challenge)
fic.close()
os.system("(openssl dgst -sha256 -hex -sign private-key.pem challenge) > signature.txt")

0

In [5]:
int(0xc3f06c42984d2967bf8828170)
int(0xffffffffffffffffffffffff1)

1267650600228229401496703205361

In [6]:
def hexa_to_decimal(C) :
    return int(C,16)
def inverse_modulaire(a,N) :
    """ int * int -> int 
        Retourne l'inverse modulaire de b dans p """
    x = a
    y = N
    (u0,v0,u1,v1) = (1,0,0,1)
    
    while y :
        
        (q,r) = divmod(x,y)
        (x,y) = (y,r)
        (u0,v0,u1,v1) = (u1,v1,u0-q*u1,v0-q*v1)
        
    return u0 % N

def reste_chinois(ni,xi) :
    
    #N
    N = 1 
    for i in ni :
        N *= i
    
    res = 0
    for i in range(0,len(ni)) :
        
        res += ( xi[i] * (N//ni[i]) * inverse_modulaire(N//ni[i],ni[i]) ) % N 
    
    return res % N

def lrack(n, k=2):
    """Racine kième entière d'un nb entier n de taille quelconque
       Génère une exception "ValueError" si n est négatif et k paire.
    """
 
    # initialisation du signe et traitement des  cas particuliers
    signe = +1
    if n < 2:
        if n < 0:
            if k % 2 == 0:
                raise ValueError("Erreur: racine paire d'un nombre négatif")
            else:
                # le calcul sera fait avec n>0 et on corrigera à la fin
                signe, n = -1, abs(n)
        else:
            return n  # ici n = 0 ou 1
 
    # trouve une valeur approchée de la racine (important pour les grds nb)
    rac1, i = n, 0  # i = compteur du nb de positions binaires utilisées
    while rac1 != 0:
        rac1 >>= 1
        i += 1
    rac1 = 1 << (i // k)
 
    # calcul de la racine en partant de la racine approchée rac1
    km1 = k - 1  # précalcul pour gagner du temps
    delta = n
    while True:
        rac2 = (km1 * rac1 + n // (rac1 ** km1)) // k
        if delta <= 1 and rac2 >= rac1:
            if signe > 0:
                return rac1
            return -rac1
        delta = abs(rac2 - rac1)  # on garde pour la prochaine boucle
        rac1 = rac2
        
def bezout_fct(a,b) :
    """ int * int -> (int,int)
        Retourne la relation de bezout entre a et b"""
    if b == 0:
        return 1,0
    else:
        u , v = bezout_fct(b , a % b)
        return v , u - (a//b)*v

In [11]:
#Attaque broadcast sur RSA
c1 = "3fbf9975c62fb1fe17f1f5b70898d214de497a7a440ff133de4ef92df6cc7655ea99369a2b08b81e77dc522ad32c66c29c783e7f9828f895eeeda4fdabc874df1eb5a789a0193432972bf84c9108a71dd14dd3ea1d1476b27f86919bca1a20753354ba73fd7e5850fc25931c9656e327f272d5a2104f7dc4d873a130ea7c6e9417720ad3c6d65a6b39f94ce5402dadfccaa5192429474c616d7c3ce2060125f162119d4ffcef2e135500fd848e8e923ec64c7df9366b24b27b6e885464c21299e5604dc47865fba4c478b93e4a356679e491586581da97c49a01e21426a1f5253b01067406e13e536611798aa0c8f82f968173d115fabaa3e59a8fe0d0e0be33"
c2 = "5b6f3a3a47bb7aca85520e972a8728b78f6ffd45583a6bd00e597cc7d5e1781e993401e84b44cdebf9633c5ea77a8e1dd6c3d8ba9a4664279a5039400db9459d4b04524c41afabc7680868ab91ec4926bca66b5154231ef11c46d8ca27e90d3a7aae8cb2789e2e0124b019377656916da385fef7c838976ce28d857ab822db3436d84b46cbeaa184624f90c203a91ca57276b140138cb109212171ccf74ca5050d50d71caad1bede4b8b6d9d6ffd23e3204b0c5ecdec79782b672e32655f3722125848bb82d2ed9b64ad182637cae10836e16e35fd5785e59b60a973f5c7074123aef765f7808d35c9df3464c6677ec53a6fe8503b61afeb2d95aaf5bb2cfdbe"
c3 = "6775ee400a44cf7874629d139f42dc27b6da9d6b0e5ee198e68fcd347774db7af8fe8d8440ada4e746ab00c09dfc8ce4bc5c3440a7a98d0f8fb264a80e2a63dda86f32af8390accbf91b5bdebde4f7f461e3f5ffff529d51dfc4abcacb7259acbc05b22161c4091c087fcc61a732dd6d9faf229dd1382f918f385f0289f888035f99f3b4fa2ec34520ce51432651e960e8b64bc811093ca926c7628d1c38e0da4b493014abcff8c1eab33aee25c868165abacbe30519fa42be27f99720fb47ea6404573e7e8fcf66aa9100fde0c66634b17d4bc56fb82f46c3be6369b47874d6d50009df414d8231b2eb053a0c67a7cf73ccda3cacd2e5f4aa0f6d6ca4b21a73"
C = [c1,c2,c3]

N_1 = 20960781819131128784421904746693803597779259819878844976612392269876152602949572240475103241020097870664683615267083600068933972066694228538418189568583189288060921694273165258133402812685827614172677011150550269067422619186392218368058733871307892475435558218654505492440537732073577572684607705551064743272876571431355385083705349691472347425829488130337179423584181299641871385640082354501404710316758219792318546761583527544656338772272921134748413080054398444170329769250362765112465645113193016971088187095808327710657547510198859393951662442689537878902050970802194064780750679227996959552278061731348627059853
N_2 = 29739874629755586520351823919003943297026927858632036311340258601107515063320068511583444746922126472602067282346281875901503609209964258205067869088405801141998711283469753561607269763155990978803594170822684206827868834273714935087066316163290346782444386853500321238039907109603121149930057082408458919269259433728497071161486159027825750052004055773368065736398163483086223003826431445181807229298164940120591494220114182770746727803897084120254042057369886355467592199897562562148238596584017048008390678982739591813234679501846515980851042348605039213737691838212375255266339305622109143502242771405955423364677
N_3 = 26215849253857368743167021313234191367279514796621658159738627341510477150213671993671251355869789290870064351716008892633324847916975582134315647500319528649250107306617290110129843734687218135153073258894850180379454262548486588884523631541461001091197628751929521942395998897275563958045781540257412619531959606923615655432877936470169884515299559985118928571186582915840382409548569459474778186550894522068297575557737564092047959560174436072461261588303675289679956400659939556212339249961747645937886859614656212909627381080056304224586388478418403062423354513387133092927754535949060170125683161267249825494299
N = [N_1,N_2,N_3]


def broadcast_attaque(C,N) :
    
    dec_C = []
    dec_N = N
    for i in range(0,len(C)) :
        dec_C.append(hexa_to_decimal(C[i]))
    
    r = reste_chinois(dec_N,dec_C)
    
    return r

In [40]:
#Attaque broadcast 2
n = 23503777373084572286838888977304942844142031153149711678630662592169255193996803887041913470344492012930077464379009460144872646728733720261187113734374277143533539802520065036351962405061670746440898504479032372090169806489266051042858364480019860411380322473716731824837779219530817613446782045006816511925480756024393326892128316164146830304588783851536610093067575175586398543982223068546162361202892504948859750133498830480844373263833868923195633912608838322257024957318126385230896234711642771564802001363303226414263655574012303556585566847032343662611450040920758262492199747662393604088463247573845296607417

e1 = 15238485794061028531923895205131368466641698349798430355486280181694333963864956011584543308873520343124443220279321559572796785848089385939899047638897380273421128465049206849710976527551226899512201500745224805512846069386001770685093379237657976271217117719146752019412946882912547939275180896520086390357964392437099941915242167534433696248678150440944227974597291294981593034390990320902884204163277729642901339318426723407786321576496528081612251301346586637571945686471304654002006582844553024569203358807400793940780845663511860289866059747308931074420693677097770806872956134966215169039086838300716096495737
e2 = 6624035335866947686058941616779049690458719080527248511672352510086072854937513078514152942088671260246075576657310734557067642562097289462253095928132621217037280816057697337483166767140257506830584462374026133187621439617620767674455740390021917021531610611562557764145621629441184071530295883323423452568857334281945026034206245315872169793195520922916557122602826986627858017871288162688759871481016553135207563725700427040120268698403740887517742605140223674071740197328394693881833404461293205372769040970457737210712457691109691743021102602702358604652733319371545989689712627229360278201282605804624732501671

c1 = "a95150f34e4895187fd967939382be1efe6f88ded7fc90ef05e39444b1f267416a0b24e32d21afe4c02aa34f9517462ea29d8cd729970fb39f0369fa936a3cf3e7920bb8ba16654391a297f149fc40e95d47b8e8da8d1874b61a5cd773f03c955fa9808ca9134fe27fef278baea2c577481d185869d923b96306b0c8aa58e265e66c5aaba4c0a6910cd5562bcdf2961d0407ceb324fc633a63ad4f19555cd14acc1e1b738a4a84bd4564f17da414b7ab2a5d7ae58774187499079ef75b236bde439ddba742de14d9545e70a1dc7ae0ea78bdc2c2206daef9748b0ee5cb869cd1c52021a47b1600d4c7c617bf847d64ee481e7efc2c88a5a6573a1bd723a9428c"
c2 = "1f629ebf7399b9ae7859b53a23eab358b7d49cb2b3d4adb5d70bd5e18167392cb4734a656c053a2384fefdd7c02a0750b6850845c8a451443375e5c904f957cd3444b0175da20f5d7ff93b77af0981da8c2ebf3b142200c4864911fc19ff96478dd75d8619e8de5378b15cfd91f9ddef55ce3783716ff2a23e636a3def6867fa5f038af218be752ef2092ae11f00cbf46f0875bffa6ef80108d43da8f13bda6ceb26b4607b99b2eafca65bbeb221839690ba14ea37cf75a9d812cc82c965c5884450ace46e9b21115ea753a4621dc99d8ba11d6800157d5dcf9bfbfe0b86b94ed947377c09722c617bbb3305366249c4a71a0597c53eadf67aa50da57c1ebee7"
c1 = hexa_to_decimal(c1)
c2 = hexa_to_decimal(c2)

def broadcast2(e1,e2,c1,c2,n) :
    u,v = bezout(e1,e2)
    if u < 0 :
        c1 = inverse_modulaire(c1,n)
        u = -u
    if v < 0 :
        c2 = inverse_modulaire(c2,n)
        v = -v
    M = ( (pow(c1,u,n)) * (pow(c2,v,n)) ) % n
    return M



In [6]:
L = [ 0xebc5cb9354dede77bce9d5164,  0xeb933dc81d555da0d393ce238,  0xcff057347bf6c39ac8dcf0a91,
          0x7b1c2d819a61c0ff9adb4ab7a,  0xf11bc5a561320a1a2880716a7,  0xdfbaf4b4bb0c6b0bc376e6d34,
          0xbf06aaadcb97b062125aefcbf,  0xfac0dc38c7f2666fabf26bd80,  0x059f47c80278bd509038c4a46,
          0xde34cb5863b26fabd20d20b66,  0x4710861c5a9027410061c5360,  0x149a07427f7dd83d3aeccf8a0,
          0xb33e1ae5112ff45f51d07ec7c,  0x9ab9963843473463ae7a7b64f,  0x402248d2702faca16e1110eca,
          0x39d90e5d614c6360f2c1a2935,  0x3e45a026d0e7b9d97fe1a2235,  0x247a1251c5e5cfaa6e29e04b6,
          0x715f608138bef43670485c693,  0x50329fd0c5bfe7b2cb95af805,  0x738c0eba0b616398abf2e6cab,
          0x207f73cfff313f22bb82e210a,  0x05653366d0997137c17dd8ca3,  0x0c082500a0402f9f6eba62de2,
          0x88a85c5d2ada778f59e0edda5,  0x9bb3c6b6df57dc0949304711a,  0xae9992d7af83a3f89b22bdfb0,
          0xe03baf80f1d21149c1c6ba19d,  0xfb8fde3daa7a1b1c3af1c6ec5,  0x73ac0b05a326a59bc6ba4ed03,
          0xb02c80157432cc5f790dc81d2,  0x83191dcd733ebd1179be009e5,  0xbdc992bbb8ecf09a6ab0fb47b,
          0x64c18f178041a1952dd20e9ec,  0xc8757aa4f2f0577536011c3ea,  0xa6a7c55387723cbccff276fd6 ]
p = 0xffffffffffffffffffffffff1
len(L)

36

In [7]:
import asn1
from hashlib import sha256

def standard_hex(chaine):
    s = ""
    for tmp in chaine:
        if (tmp != ":") and (tmp != " "):
            s += tmp
    return s

def verif_sign(g,q,p,h,s,c,msg):
    
    #On teste si on a bien récupéré les bons g, q et p
    pui = pow(g,q,p)
    assert pui == 1

    #On teste si la fonction inverse fonctionne bien
    assert (pow(h,-1,p) * h) % p == 1

    #On calcule r = (g^s * h^(-c)) mod p
    r = ( pow(g,s,p) * pow(pow(h,-1,p),c,p) ) % p
    assert 0 < r < p
    assert pow(h,c,p) == pow(h,c,p)
    assert pow(pow(h,-1,p),c,p) == pow(pow(h,c,p),-1, p)

    #On vérifie que c == SHA256(M || r)
    msg_bytes = msg.encode()
    r_bytes = (r).to_bytes(length=256, byteorder='big')
    hashedWord = sha256(msg_bytes+r_bytes).digest()
    
    assert hashedWord == c.to_bytes(length=32,byteorder='big')
    return True

def find_x_key(s1,c1,s2,c2,p,g,h,q):
    #Protocole : s  <-- k + c  * x mod q
    #Protocole : s' <-- k + c' * x mod q
    # --> x = (s-s')/(c-c') mod q
    inv_diff = pow(c1-c2,-1,q)
    x = ((s1-s2)*inv_diff) % q
    ### VERIFICATIONS ###
    assert pow(g, q, p) == 1
    assert inv_diff == pow(c1-c2,-1,q)
    assert (inv_diff * (c1-c2)) % q == 1
   
    return x

def find_k_key(s,c,x,q):
    #Protocole : s <-- k + c * x mod q
    k = (s - c*x) % q
    return k

def digicode_cha(challenge,k,g,p,x,q):
    r = pow(g,k,p)
    cha_bytes = challenge.encode()
    r_bytes = (r).to_bytes(length=256, byteorder='big')
    c_hex = sha256(cha_bytes+r_bytes).hexdigest()
    c = int(c_hex,16)
    s = (k + c*x) % q
    # Verification
    verif_sign(g,q,p,h,s,c,challenge)
    # Convertion en signature
    encoder = asn1.Encoder()
    encoder.start()
    encoder.write(c, asn1.Numbers.Integer)
    encoder.write(s, asn1.Numbers.Integer)
    encoded_bytes = encoder.output()
    sign = "3045" + encoded_bytes.hex() # 3045 == Séquence
    return sign

### Ce que vous devez changer ###

# On extrait la clé publique de Mohab (Cmd bash : openssl dsa -in cle-publique-mohab.pem -pubin -text -noout)
h = "42:55:5a:90:b4:1a:d2:60:d8:e5:e7:a0:14:b8:e9:cb:c3:e8:1a:b4:b9:25:58:8b:75:9a:f7:4a:8f:1f:b2:27:13:6a:9c:9e:29:3c:16:a2:c9:b9:83:12:a3:ce:95:69:ee:c4:28:ea:fa:ba:9c:e8:de:1f:86:19:81:74:16:2c:1a:0d:79:2d:25:e3:a6:a7:2b:34:ad:f9:ce:33:3a:63:07:c4:b0:cc:24:7f:85:ac:03:61:fe:f7:f7:cc:8d:f2:7b:be:ea:ae:a4:5d:00:c5:64:cd:69:82:d5:db:81:0a:ea:3d:4e:2c:de:88:32:b7:ab:f8:47:38:16:9c:3e:97:29:1f:6c:65:83:bf:e3:22:20:8d:c2:45:06:1e:f1:4c:ff:a1:45:c3:29:3b:61:a7:45:9d:df:bb:64:22:3e:c4:2d:63:a1:2b:d4:8f:09:fe:4a:c3:70:64:de:2c:25:a7:5e:b1:31:7a:8d:a6:c3:20:dc:4b:79:87:30:b4:79:b7:af:a5:29:f3:66:4f:a9:29:84:d4:0d:a7:0e:48:d0:27:24:b4:db:69:5e:e6:ae:f3:ab:66:40:6f:a7:3d:19:24:1f:5c:7c:89:57:49:bd:b8:b9:1e:88:47:db:5c:03:69:9d:db:c0:0c:2b:92:c7:16:5d:86:9e:a5:dd:95:dd:88"
p_hex = "00:fe:4e:b9:97:ef:43:25:2b:95:50:4c:1f:f7:07:21:da:8a:90:a9:e8:de:42:40:a4:f9:f9:63:97:70:f2:20:5d:4d:3e:7f:7a:dd:8f:a4:ef:7e:5b:d8:64:e6:9a:42:3e:77:a1:54:03:3a:0e:05:11:b7:3c:cb:b9:9f:63:3e:cb:a0:01:85:6d:79:ef:ca:5c:63:6d:18:cb:02:f0:a7:c3:83:5f:a9:a1:4f:b8:e2:e3:57:e9:3d:e3:c3:a4:8d:c9:ea:1a:47:8d:1b:af:97:4b:10:d9:80:d8:d7:62:cf:b9:e7:e3:c3:ee:7b:17:01:12:dd:c0:bf:e4:ee:12:9f:63:c6:ed:d9:b2:f0:ef:b1:97:f8:0e:8b:04:83:82:cf:13:ea:75:5d:6f:4b:03:b9:b4:58:8b:06:35:f2:8e:8d:ba:03:4d:11:b8:a2:35:c5:7e:01:7b:e9:d2:76:df:14:fb:67:42:b3:9e:53:ce:fa:b5:66:6d:74:08:a0:8c:0a:22:78:04:53:9b:80:73:fa:1b:b2:55:b9:71:f0:b3:16:f2:84:75:9a:17:46:b8:d3:57:b5:21:3b:09:78:8d:65:0e:86:06:92:b1:65:35:24:02:51:dc:5e:a3:51:a4:cf:63:7e:dc:94:26:d1:c3:e5:81:10:e6:25:71:e2:48:38:cd"
q = "00:ef:f0:58:5c:29:80:f0:d8:3c:44:a7:ea:5d:98:7f:0e:cd:01:78:36:0f:23:70:61:82:cd:30:a4:86:ce:a9:8b"
g = "00:97:10:74:ef:3a:e8:c8:29:1f:dc:8e:23:4f:6c:c8:02:aa:4c:0e:4a:dd:0b:f9:87:21:89:56:ad:50:6a:c6:18:69:b1:ef:74:8e:33:0a:60:ab:ae:8d:9e:bc:8a:d4:e4:53:00:88:12:cc:79:29:e5:03:44:02:1a:10:fb:12:45:3d:d8:63:7f:81:f2:7b:6d:84:a4:4f:11:0d:fc:a0:93:d6:79:5c:04:9a:56:c9:10:d9:05:c6:c1:26:62:86:66:33:e8:d1:02:03:43:e6:3d:26:32:6c:19:c9:64:f5:9b:63:19:5a:d7:2c:3a:b7:83:f4:10:53:7d:38:29:7c:62:16:ce:c6:c8:2f:b4:b2:e9:c2:2c:33:1f:8d:76:3c:be:c3:b0:00:2d:0c:80:d7:a7:19:db:08:4f:68:a8:89:14:53:15:79:d5:9d:ee:3b:10:c5:6d:f7:2f:0c:b3:03:25:d5:2b:fd:68:ac:86:8d:88:4d:e3:b6:bd:cf:6b:99:0b:8c:b8:45:55:54:d6:cf:ef:ca:62:c6:e3:6e:06:5b:a6:3b:cf:99:83:3b:7d:c7:74:ef:d1:af:ec:82:62:36:d7:fb:65:67:d4:c1:e9:35:f3:b9:81:f1:73:bb:ac:9a:ee:56:ca:33:dd:fa:25:d8:02:29:74:cc:16:92:0a:67:ab"


# Signature de l'email 1 de Mohab (ASN.1 decoder : https://holtstrom.com/michael/tools/asn1decoder.php)
c_hex = "4741E707412BC88B485B1EA2C684684905C31D628AA9BFB1D34489778CB7DBE2"
s_hex = "47D51BC0C61C53333DEE79C183F37A32C0AD51FA037BFA6C37AF9109899C4240"

# Signature de l'email 2 de Mohab
c_hex2 = "C478E8DD7AF41E2389472DFFA5168DD312DF94A24E83AE54BE9708DE794E67FB"
s_hex2 = "65023A521BF10E06DFA713C1086BD6593A6BFC3535FCF2BB6E54EE866E497AE4"

# Le challenge à récupérer après avoir mis le mail de Mohab dans le digicode
challenge = "wraps ploys patsy duddy bents"

### On change plus rien après et on exécute ###

h = int(standard_hex(h),16)
p = int(standard_hex(p_hex),16)
q = int(standard_hex(q),16)
g = int(standard_hex(g),16)
c1 = int(c_hex,16)
s1 = int(s_hex,16)
c2 = int(c_hex2,16)
s2 = int(s_hex2,16)

msg = """Bonjour Yanny,
As-tu bien prévenu les étudiants qu'ils doivent quitter le campus ?
Merci,
---
Mohab"""

#Vérif 1ere signature
verif_sign(g,q,p,h,s1,c1,msg)

msg2 = """PS : a-t-on beaucoup de pertes parmi les étudiants ?
---
Mohab"""

#Vérif 2eme signature
verif_sign(g,q,p,h,s2,c2,msg2)

# On récupère le clé secrète x et le k
x = find_x_key(s1,c1,s2,c2,p,g,h,q)
k = find_k_key(s1,c1,x,q)

print("Digicode : " + digicode_cha(challenge,k,g,p,x,q))

Digicode : 3045022100c13e497ab1adb2e5ec7f425241408ad8b1a9bf667376d7724a1e5a251e5065ae022100e2595e12ba9c82fde51b80a64449cc948218492d3678a97c4c3d93c019aafb10
